In [1]:
# ---- Nested function ---- #
import os
import pandas as pd
import wfdb
import ast
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
from pprint import pprint
from collections import Counter
import math
from copy import deepcopy
import random

# # ---- BWR ---- #
# import bwr
# import emd
import pywt
# ---- Scipy ---- #
from scipy import signal
from scipy.signal import butter, lfilter, freqz, filtfilt
from scipy.fftpack import fft
from scipy.signal import find_peaks
from scipy.interpolate import interp1d


# ---- PyTorch ---- #
import torch
import torchvision
import torch.nn.functional as Func
from torch import nn
from torch import optim
from torch import functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision.transforms import ToTensor
from torch.nn.functional import softmax
from torch.nn.parallel import DistributedDataParallel
from pytorchtools import EarlyStopping
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import torchvision.ops as ops
import tensorboard
from tensorboardX import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau,CosineAnnealingLR,CosineAnnealingWarmRestarts

# ---- Scikit Learn ---- #
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import KFold


# ---- Matplotlib ---- #
import matplotlib.pyplot as plt
import seaborn as sns

# ---- Summary ---- #
import pytorch_model_summary


In [23]:
input_path = "/data/ECG_DATASET/large_scale/"
records = open(input_path+"RECORDS","r")
records_list = []
for l in records:
    l = l.rstrip()
    records_list.append(l)

In [24]:
records_list

['WFDBRecords/01/010/',
 'WFDBRecords/01/011/',
 'WFDBRecords/01/012/',
 'WFDBRecords/01/013/',
 'WFDBRecords/01/014/',
 'WFDBRecords/01/015/',
 'WFDBRecords/01/016/',
 'WFDBRecords/01/017/',
 'WFDBRecords/01/018/',
 'WFDBRecords/01/019/',
 'WFDBRecords/02/020/',
 'WFDBRecords/02/021/',
 'WFDBRecords/02/022/',
 'WFDBRecords/02/023/',
 'WFDBRecords/02/024/',
 'WFDBRecords/02/025/',
 'WFDBRecords/02/026/',
 'WFDBRecords/02/027/',
 'WFDBRecords/02/028/',
 'WFDBRecords/02/029/',
 'WFDBRecords/03/030/',
 'WFDBRecords/03/031/',
 'WFDBRecords/03/032/',
 'WFDBRecords/03/033/',
 'WFDBRecords/03/034/',
 'WFDBRecords/03/035/',
 'WFDBRecords/03/036/',
 'WFDBRecords/03/037/',
 'WFDBRecords/03/038/',
 'WFDBRecords/03/039/',
 'WFDBRecords/04/040/',
 'WFDBRecords/04/041/',
 'WFDBRecords/04/042/',
 'WFDBRecords/04/043/',
 'WFDBRecords/04/044/',
 'WFDBRecords/04/045/',
 'WFDBRecords/04/046/',
 'WFDBRecords/04/047/',
 'WFDBRecords/04/048/',
 'WFDBRecords/04/049/',
 'WFDBRecords/05/050/',
 'WFDBRecords/05

In [25]:
file_list = []
for rec in records_list:
    record_path = open(input_path+rec+"RECORDS","r")
    for fi in record_path:
        fi = fi.rstrip()
        file_list.append(input_path+rec+fi)

In [58]:
labels = [164873001, 89792004, 446358003, 164931005, 164930006, 429622005, 428750005, 426783006, 164865005]

real_record = []
failed_record = []
for j, file in enumerate(file_list):
    try:
        record = wfdb.rdrecord(file)
    except Exception:
        failed_record.append(file)
        continue
    
    labels_index = {snomed: i for i, snomed in enumerate(labels)}
    metadata = dict([i.split(": ") for i in record.comments])
    y = np.zeros(len(labels_index))
    if "Dx" in metadata:
        snomeds = map(int, metadata["Dx"].split(","))
        indices = [labels_index[i] for i in snomeds if i in labels_index]
        if len(indices) == 0:
            continue
        y[indices] = 1
    real_record.append({"file": file, "y": y})

print(len(real_record))

11713


In [35]:
# 164865005 : MI

Sinus Rhythm
# 426783006 : SR

ST
# 428750005 : STTC
# 429622005 : STDD
# 164930006 : STE
# 164931005 : STTU

HYP
# 446358003 : RAH
# 89792004 : RVH
# 164873001 : LVH

In [54]:
len(real_record)

45150